In [2]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [6]:
base_dir = '/content/Wound_dataset'
train_dir = '/content/train'
test_dir = '/content/test'

In [9]:
# Load the VGG16 model as the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [11]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(3, activation='softmax')(x)

In [12]:
model = Model(inputs=base_model.input, outputs=output)

In [14]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 53 images belonging to 3 classes.


In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 56 images belonging to 3 classes.


In [ ]:
# Train the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 47s/step - accuracy: 0.4256 - loss: 2.4408 - val_accuracy: 0.5893 - val_loss: 0.8765
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 41s/step - accuracy: 0.4707 - loss: 1.9136 - val_accuracy: 0.5536 - val_loss: 0.8977
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 82s 47s/step - accuracy: 0.5804 - loss: 1.3502 - val_accuracy: 0.5893 - val_loss: 0.9074
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 61s 49s/step - accuracy: 0.4573 - loss: 1.4879 - val_accuracy: 0.5714 - val_loss: 0.9577
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 42s/step - accuracy: 0.6026 - loss: 1.0519 - val_accuracy: 0.7143 - val_loss: 0.8566
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 82s 42s/step - accuracy: 0.6612 - loss: 0.8514 - val_accuracy: 0.6429 - val_loss: 0.8988
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 59s 47s/step - accuracy: 0.6274 - loss: 0.8277 - val_accuracy: 0.6607 - val_loss: 0.8790
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 60s 47s/step - accuracy: 0.6466 - loss: 0.9829 - val_accuracy: 0.6786 - val_loss: 0.8443


In [18]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.2f}')
print(f'Test accuracy: {accuracy:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 30s 13s/step - accuracy: 0.6116 - loss: 0.8687
Test loss: 0.88
Test accuracy: 0.59
